# MERGING AND CLEANING DATA 
## fact_nps, customer and application_analytics tables

## Importing libraries and initial config

In [2]:
import numpy as np
import pandas as pd
import scipy as scs
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
import datetime as dt
import re

import gensim
from gensim.utils import simple_preprocess
import nltk

import gensim.corpora as corpora
from pprint import pprint

#import pyLDAvis.gensim_models
import pickle 
#import pyLDAvis

pd.set_option("display.max_columns", 50, "display.max_rows", 100)

In [3]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Reading in table: _fact_nps_

In [4]:
sql = """
    WITH A AS
      (SELECT * FROM `pg-duke-student-capstone-v1.source_data.fact_nps`)
      SELECT * FROM A
      -- limit 10
"""

# Run a Standard SQL query with the project set explicitly
project_id = 'pg-duke-student-capstone-v1'
df1 = pd.read_gbq(sql, project_id=project_id, dialect='standard')
df1.head()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=z2z4dZxJhLcCOE2jQiSz6e3rtoOJqx&prompt=consent&access_type=offline
Enter the authorization code: 4/1ARtbsJocZRGQ262EPADuWqjF1gfQs9f3zkaQ0esqfG4XuxkXMB370oRJj2Q


,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim
0,NaN,None,480333851,k7duvQF3rRN3UwPQ0wHYUnYjWuVVeL31,2019-11-05 21:40:52,0,NaN,None,Prices for policies are too high.
1,NaN,None,430274902,sgL7fhPhywxhnA6NNk17re9pMLw6MioL,2019-08-03 11:56:04,0,NaN,None,I thought I would receive a quote for the prod...
2,NaN,None,559903515,BfRJUFj4X0fnvYf7l4Yrh9nB89Xohyjo,2020-02-19 14:13:46,0,NaN,None,Turnaround time on initial quote was so slow I...
3,NaN,None,434854035,VU4sdXaXt8GyY71Aze3LvQLO4wZBUpJG,2019-08-13 22:08:32,0,NaN,None,Why bother advertising in an area that you don...
4,NaN,None,421691454,9tfZ9TQoWc9V6Yc517I1JYcILYO6BL5l,2019-07-17 12:46:09,0,NaN,None,There was no help provided because I am buying...


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27066 entries, 0 to 27065
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   application_id     24683 non-null  float64       
 1   product_type       24683 non-null  object        
 2   respondent_id      27066 non-null  object        
 3   survey_key         27066 non-null  object        
 4   created_at         27066 non-null  datetime64[ns]
 5   score              27066 non-null  int64         
 6   zendesk_ticket_id  2415 non-null   float64       
 7   medium             2415 non-null   object        
 8   verbatim           15288 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.9+ MB


## EDA: _fact_nps_

### Checking for duplicates

In [6]:
# checking for duplicates in application id
df1.loc[df1.duplicated(keep=False)].sort_values(by="application_id")

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim
4103,279510.0,homeandauto,612190848,iUkMRMIkVwAwZIXoackjT3eHzmUkKVye,2020-05-09 13:06:54,10,NaN,None,Great experience so far
1885,279510.0,homeandauto,612190848,iUkMRMIkVwAwZIXoackjT3eHzmUkKVye,2020-05-09 13:06:54,10,NaN,None,Great experience so far
13529,296671.0,homeandauto,611547555,N4SH3KMfBCAF0TIMjNZoL9ALjnDttvZC,2020-05-08 15:27:47,9,NaN,None,"So far, the customer service has been good. I ..."
2442,296671.0,homeandauto,611547555,N4SH3KMfBCAF0TIMjNZoL9ALjnDttvZC,2020-05-08 15:27:47,9,NaN,None,"So far, the customer service has been good. I ..."
4136,296843.0,homeandauto,611283396,cVyQDL7l2Jhmb7rSmlYQqIEiHws8id5P,2020-05-08 09:54:50,10,NaN,None,So easy to use and I'm confident that i'm gett...
6961,296843.0,homeandauto,611283396,cVyQDL7l2Jhmb7rSmlYQqIEiHws8id5P,2020-05-08 09:54:50,10,NaN,None,So easy to use and I'm confident that i'm gett...
22606,298952.0,homeandauto,611415537,PGWvV3BtWG9Z27nMbISSd2LMOVMukt9v,2020-05-08 12:47:15,9,NaN,None,None
23441,298952.0,homeandauto,611415537,PGWvV3BtWG9Z27nMbISSd2LMOVMukt9v,2020-05-08 12:47:15,9,NaN,None,None
21957,316475.0,homeandauto,611437558,6YrfLndCtHtYRqzo67oQN7NSDNXIx4gM,2020-05-08 13:17:00,8,NaN,None,None
24902,316475.0,homeandauto,611437558,6YrfLndCtHtYRqzo67oQN7NSDNXIx4gM,2020-05-08 13:17:00,8,NaN,None,None


In [7]:
# dropping these records
df2 = df1.loc[~df1.duplicated(keep="first")].copy()
df2.shape

(27023, 9)

In [9]:
# Verify to see if the duplicates were dropped [Looks like they were dropped successfully]
df2.loc[df2.duplicated(keep=False)]

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim


In [10]:
# checking for remaining multiple application ids [These time they look unique by product type, so we dont drop]
df2.loc[df2.duplicated("application_id", keep=False)].sort_values("application_id").head(10)

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim
13267,397684.0,homeandauto,663506789,goLwg0B1NEwCoIFxrWJAl5c9cHi1R3lb,2020-07-16 12:31:56,10,NaN,None,Competitive prices and great service.
22397,397684.0,disability,95159610,jk3ooncSHqUoJcSgowGoBSy0LGbgWfw4,2021-06-17 18:16:44,10,NaN,None,None
14743,536917.0,life,129270378,Z4u1NbOQs2mm0ZuTkdYAnDDqninHIGZI,2017-05-09 22:29:46,9,134803.0,Web Service,They really made it easy. I wish ended up with...
5230,536917.0,homeandauto,728041086,qHb3FE1aQv6jlqw3omEuQi0QCvNXD7fI,2020-10-09 19:05:16,10,NaN,None,They saved me a great deal on my auto insurance.
21743,538474.0,homeandauto,714634085,MNdDLxmQ0kjZcUjc7rsI4pNrrWPHbbYv,2020-09-22 13:08:31,10,NaN,None,None
26066,538474.0,life,120161045,wUmA7JEQqh2M98QlS6SBs78qm1enefqa,2017-03-28 21:23:12,10,136274.0,Web Service,None
24077,598084.0,homeandauto,754183059,J9o6Bh15PrB3t8ubwYNBO0QsXp4nBY3e,2020-11-13 10:07:07,10,NaN,None,None
25965,598084.0,life,159620196,WYNlphm2D5G8TYNZylMAxVYsncBe5rbX,2017-10-01 22:04:21,9,171167.0,Web Service,None
488,637110.0,homeandauto,782672452,m3fPAkYNuBpMmmIVHRf1FnO8HT7cTtKx,2020-12-18 11:46:48,8,NaN,None,"It was helpful, but I don't have enough long t..."
14929,637110.0,life,215831818,VUfyruoxyPJC3UKOnA9HaykPh79oKZxz,2018-03-13 18:50:51,9,196340.0,Web Service,Excellent service


In [11]:
## checking the tail [We notice nas in application id]
df2.loc[df2.duplicated("application_id", keep=False)].sort_values("application_id").tail(10)

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim
27054,NaN,None,50160232,Ob4LLIQVCeQHuHNTtknLy5sZawGNRhpw,2017-10-12 15:21:24,9,191702.0,Closed Ticket,None
27056,NaN,None,128781292,EI4Q4wphaX9Hu8z39ACuFAtJVmxrOzDV,2017-10-31 15:19:17,0,196023.0,Closed Ticket,None
27057,NaN,None,142399645,5M9dt6g2Z8l4S3MSoacxhtskXA3rtsOc,2018-05-13 06:45:25,10,304885.0,Closed Ticket,None
27058,NaN,None,204210572,xwFGzWNEHsRCoyvI3JV8Ve7Y914NfUtX,2018-02-07 08:30:47,10,264281.0,Closed Ticket,None
27060,NaN,None,128781382,YpoGFEKXeoQ1kf5Swjh1puoFxpPduPXS,2017-05-07 13:48:08,10,130479.0,Closed Ticket,None
27061,NaN,None,218582481,67FP6UpEPtkxtwWR45U836E9OuNPId0x,2018-03-20 19:52:08,0,283990.0,Closed Ticket,None
27062,NaN,None,98124394,jr8rIwoP220x1uIXqNcImtLXPujhk61H,2017-04-19 12:42:20,9,150550.0,Closed Ticket,None
27063,NaN,None,144188736,ljZC5rSjRu3Huwu4EJfOPClBTTA5t5uW,2018-03-24 21:30:31,10,238427.0,Closed Ticket,None
27064,NaN,None,188875882,Gv1sE4tuY1oGCIAk5L9U76ub1hYgJWbs,2017-12-24 10:48:09,9,204029.0,Closed Ticket,None
27065,NaN,None,116126928,RYiF7naZVk2n9XVcIewMrin8f2XHIFtl,2017-06-13 16:12:32,0,158542.0,Closed Ticket,None


There are still multiple responses per application id, these are generally for different products. Can check uniqueness by respondent_id and/or survey_id.

In [13]:
## Checking uniqueness by respondent id [seems unique by the different time points, dont drop]

df2.loc[df2.duplicated("respondent_id", keep=False)].sort_values("respondent_id").head(10)

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim
15238,679998.0,life,120844810,rL0Hd8rxrxD7RlkyxIjV9zTn0DZCNU25,2018-03-15 12:54:45,9,260402.0,Web Service,"Good and friendly service, worked well with me..."
14803,530197.0,life,120844810,n1bzqcQvHj8XlHiPyw3FNZrOFngOxGBH,2017-04-12 21:07:48,10,130280.0,Web Service,"Easy process, good customer service."
15260,NaN,None,120954818,2jizsCrXmnzcBoozaQphMjwQf8ARtrTK,2017-07-25 15:58:08,5,157581.0,Closed Ticket,The customer service was atrocious. Perhaps th...
26820,515285.0,life,120954818,2w0M8ab5BaN0gJpS8Yi2v92hzRyAmPdG,2017-05-22 10:03:44,3,124075.0,Web Service,None
4767,3589541.0,life,127139476,R2infD7CbJT787PXhfU06DUtCKODPHlh,2021-03-25 09:23:56,10,NaN,None,Second time using Policy Genius and customer s...
14672,31734.0,disability,127139476,zZyBrUEpTyzt524Bwjki8cGHUgbHTzcV,2017-04-28 14:07:31,10,131702.0,Web Service,Loved my experience. Tyler was a pleasure to w...
26742,505324.0,life,128781292,7AoFVLsoJoRUEVnnebchZZZdIyTlPRIv,2017-05-06 17:20:43,8,118906.0,Web Service,None
27056,NaN,None,128781292,EI4Q4wphaX9Hu8z39ACuFAtJVmxrOzDV,2017-10-31 15:19:17,0,196023.0,Closed Ticket,None
14407,537203.0,life,128785110,gDs8GPVJ5uTMghFCULqcEHarlhzrR40G,2017-05-06 11:20:36,0,134850.0,Web Service,You don't return phone calls in a timely manner
16160,931687.0,life,128785110,UMr1lTS7cvf0JAMD1j12IXOvqxOXsOWQ,2018-08-15 11:02:53,0,NaN,None,None


Instances with same respondent_id but different application_id. Survey_key still seems unique for these records.

In [14]:
## Srurvey id is duplicated by these few observations, have to make a decision to drop one. 
## Since these cases are only four, we can drop one per duplicated survey id

df2.loc[df2.duplicated("survey_key", keep=False)].sort_values("survey_key")

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim
14543,635198.0,life,213415883,9brA1HsHElwacAPVfFtzi6MwX4skMLgd,2018-03-07 09:10:21,5,195487.0,Web Service,Took 3-4 months to finalize
14924,646634.0,life,213415883,9brA1HsHElwacAPVfFtzi6MwX4skMLgd,2018-03-07 09:10:21,5,195487.0,Web Service,Took 3-4 months to finalize
26241,15353.0,disability,157436523,ktktyl5QubKnFMBQTYnG8ifKXJ3J0jY8,2018-05-09 19:56:59,10,206774.0,Web Service,None
26713,61006.0,disability,157436523,ktktyl5QubKnFMBQTYnG8ifKXJ3J0jY8,2018-05-09 19:56:59,10,206774.0,Web Service,None


Jackpot?! At the level of survey_key, all records are unique, except the above, which we can drop.

In [15]:
## Keeping the first uniue survey id from above
df3 = df2.loc[~df1.duplicated("survey_key", keep="first")].copy()
df3.shape

(27021, 9)

In [16]:
## Now confirm to see any duplicates by survey id again [No , duplicates as seen]
df3.loc[df3.duplicated(subset = ["survey_key","verbatim"], keep=False)]

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim


In [18]:
## checking number of records after dealing with application, respondent, [we have 24638 records in total]
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27021 entries, 0 to 27065
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   application_id     24638 non-null  float64       
 1   product_type       24638 non-null  object        
 2   respondent_id      27021 non-null  object        
 3   survey_key         27021 non-null  object        
 4   created_at         27021 non-null  datetime64[ns]
 5   score              27021 non-null  int64         
 6   zendesk_ticket_id  2413 non-null   float64       
 7   medium             2413 non-null   object        
 8   verbatim           15262 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 2.1+ MB


At this stage, even with multiple application IDs per respondent_id or vice-versa, we end up with a unique number of survey responses.

### Checking for nulls

In [19]:
df3_nulls = pd.DataFrame({"percent_missing" : df3.isna().sum()*100/len(df3)})
df3_nulls

,percent_missing
application_id,8.819067
product_type,8.819067
respondent_id,0.000000
survey_key,0.000000
created_at,0.000000
score,0.000000
zendesk_ticket_id,91.069909
medium,91.069909
verbatim,43.518005


In [20]:

#checking for nans in product type
df3["product_type"].value_counts(dropna = False)

life           20947
homeandauto     3245
NaN             2383
disability       446
Name: product_type, dtype: int64

In [ ]:
# checking value counts for product_type
#pd.DataFrame(df3.groupby("product_type", dropna = False).size().sort_values(ascending = False), columns = ["counts"])

We would ideally like to keep _product_type_, could check what the records with missing product types look like.

In [21]:
## Checking missingness in product type
df3.loc[df3["product_type"].isna()].head(10)

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim
0,NaN,None,480333851,k7duvQF3rRN3UwPQ0wHYUnYjWuVVeL31,2019-11-05 21:40:52,0,NaN,None,Prices for policies are too high.
1,NaN,None,430274902,sgL7fhPhywxhnA6NNk17re9pMLw6MioL,2019-08-03 11:56:04,0,NaN,None,I thought I would receive a quote for the prod...
2,NaN,None,559903515,BfRJUFj4X0fnvYf7l4Yrh9nB89Xohyjo,2020-02-19 14:13:46,0,NaN,None,Turnaround time on initial quote was so slow I...
3,NaN,None,434854035,VU4sdXaXt8GyY71Aze3LvQLO4wZBUpJG,2019-08-13 22:08:32,0,NaN,None,Why bother advertising in an area that you don...
4,NaN,None,421691454,9tfZ9TQoWc9V6Yc517I1JYcILYO6BL5l,2019-07-17 12:46:09,0,NaN,None,There was no help provided because I am buying...
5,NaN,None,537895008,3DVjt4ucFO8k1btH4zwnqpPqE3hsEOqA,2020-01-23 13:22:19,0,NaN,None,Such a huge waste of my time. Took weeks to re...
6,NaN,None,533583725,BmY9vIPNsLHUJO2HYV1I4hRNNOkSL54i,2020-01-17 13:52:13,1,NaN,None,Took far too long to get any prices. Only rece...
7,NaN,None,508282566,JZmE9QKdVl4tXsTpRMoCT1XxU5hHd3fc,2019-12-12 12:51:35,2,NaN,None,They did not give me correct information in th...
8,NaN,None,497136189,1quLzRHdsOQrqAoVIQS8Lt04eWJkQH6a,2019-11-29 11:54:31,2,NaN,None,After going through the motions of filling out...
9,NaN,None,520524400,POvEkPFPfa8N8YW9eroN1FPC2K3L1LRj,2019-12-31 14:14:47,2,NaN,None,It took a long time to get a quote and the quo...


In [22]:
## checking where product type is na and application id is not na [they are missing together]
df3.loc[(df3["product_type"].isna()) & (df3["application_id"].notna())]

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim


In [ ]:
## Our data with resolved duplicates
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27021 entries, 0 to 27065
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   application_id     24638 non-null  float64       
 1   product_type       24638 non-null  object        
 2   respondent_id      27021 non-null  object        
 3   survey_key         27021 non-null  object        
 4   created_at         27021 non-null  datetime64[ns]
 5   score              27021 non-null  int64         
 6   zendesk_ticket_id  2413 non-null   float64       
 7   medium             2413 non-null   object        
 8   verbatim           15262 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 2.1+ MB


This tells us application_id and product_type are always nulls simultaneously. So to include all applications, we would have to bunch together all product types under one analysis, or drop the records with null product_type if we want a separate analysis per product.

TODO: also keep a separate dataset without dropping null _application_ids_.

Moving forward with the latter option for now.

### OBTAINING DATA FOR VERBATIM BIG ANALYSIS (N = 15262)
### Note that we drop about 38% of the data to get 15262 verbatim records

In [24]:
# Dropping all records where verbatim is missing
df3.dropna(subset = ["verbatim"], inplace = True)

In [25]:
df3.shape

(15262, 9)

In [26]:
df3_verbatim = df3.copy()
df3_verbatim.shape          ## use this is our verbatim big data for analysis

(15262, 9)

#### Now dropping missing application ids, we are left with (N = 13261)

#### Note that we drop about 13% of the above data to give 13261 records with no missing application id

#### We do this to obtain demographic information for the application ids

In [27]:
# dropping missing application ids, also note that missing product id = missing application_id 
#df3.dropna(subset = ["product_type"], inplace = True)
df3.dropna(subset = ["application_id"], inplace = True)


In [29]:
## checking if the application ids were successfully dropped [No missing application id now]
df3_nulls = pd.DataFrame({"percent_missing" : df3.isna().sum()*100/len(df3)})
df3_nulls

,percent_missing
application_id,0.000000
product_type,0.000000
respondent_id,0.000000
survey_key,0.000000
created_at,0.000000
score,0.000000
zendesk_ticket_id,91.810572
medium,91.810572
verbatim,0.000000


In [30]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13261 entries, 40 to 15282
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   application_id     13261 non-null  float64       
 1   product_type       13261 non-null  object        
 2   respondent_id      13261 non-null  object        
 3   survey_key         13261 non-null  object        
 4   created_at         13261 non-null  datetime64[ns]
 5   score              13261 non-null  int64         
 6   zendesk_ticket_id  1086 non-null   float64       
 7   medium             1086 non-null   object        
 8   verbatim           13261 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.0+ MB


In [32]:
# checking nulls in the remaining columns
#for col in ["zendesk_ticket_id", "medium"]:
#  display(pd.DataFrame(df3.groupby(col, dropna = False).size().sort_values(ascending = False), columns = ["counts"]))

Ignoring these for now, doesn't seem relevant for our analysis.

In [34]:
# We generally have 13253 unique application ids

app_id_ls = df3["application_id"].unique().tolist()
len(app_id_ls) 

13253

In [35]:
## checking duplicates in application ids before merging with the rest [we are okay with application and product type uniquness]

df3.loc[df3.duplicated(["application_id", "product_type"], keep=False)].sort_values("application_id").tail(10)


,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim


## Importing features from the other tables

### Reading in table: _application_analytics_user_id_mapping_ and _customer_

#### First merge application_analytics table (left) with customer table (right) on "analytics_user_id" using left join

#### Make sure we dont have any nulls in analytics user ids. This will ensure all nulls removed to avoid longer time with joins involving nulls.

#### We target for 2675953 records if the merge completes successfully. These are the total records with complete application ids in the application_analytics table

In [36]:


sql = """
    WITH map_cust AS
      (select * from `pg-duke-student-capstone-v1.source_data.application_analytics_user_id_mapping` as left_t
      LEFT JOIN `pg-duke-student-capstone-v1.source_data.customer` as right_t
      on left_t.analytics_user_id = right_t.analytics_user_id
      where left_t.analytics_user_id is not null -- redundant, no nulls for analytics_user_id
      )
      SELECT * FROM map_cust
"""

project_id = 'pg-duke-student-capstone-v1'
df_map = pd.read_gbq(sql, project_id=project_id, dialect='standard')
df_map.shape

(2675953, 16)

In [37]:
## check the data after the merge between customer table and application analytics table

df_map.head()


,application_id,analytics_user_id,product_type,analytics_user_id_1,gender,state_code,individual_income,age,marketing_application_source,first_touch_device,bmi,num_conditions,history_of_tobacco_use,marital_status,segment,segment_description
0,2944055,c9d523af6443407dadb016fcadd64189,life,c9d523af6443407dadb016fcadd64189,male,CA,225000.0,29.0,Direct,Desktop,23.0,1.0,True,married,None,None
1,2054634,7a2d685dce464915acee01c29f4b7918,life,7a2d685dce464915acee01c29f4b7918,male,NY,NaN,44.0,Direct,Desktop,35.0,0.0,True,None,None,None
2,857309,8bd1208f6018460eaf71ac0c6f4e6d7e,life,8bd1208f6018460eaf71ac0c6f4e6d7e,None,None,NaN,NaN,Direct,Desktop,NaN,0.0,None,None,None,None
3,3618040,05acb3fab073455c9cc67069bd9f004a,life,05acb3fab073455c9cc67069bd9f004a,male,GA,70000.0,65.0,Direct,Desktop,27.0,1.0,False,None,None,None
4,304911,49504ccadd70ac42a47b74fbd038eb15,life,49504ccadd70ac42a47b74fbd038eb15,male,PA,1451.0,73.0,Direct,Desktop,29.0,2.0,False,None,None,None


## Check duplicates in application id. 
## Again we have unique by application id ad product type.
## Meaning we can merge on application id and product type just fine.

In [47]:
df_map.loc[df_map.duplicated(["application_id", "product_type"], keep=False)].sort_values(["application_id"]).head(10)

,application_id,analytics_user_id,product_type,analytics_user_id_1,gender,state_code,individual_income,age,marketing_application_source,first_touch_device,bmi,num_conditions,history_of_tobacco_use,marital_status,segment,segment_description
42690,1655,5b20e9622c2e49c186dd6b668620240d,homeandauto,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None
1929916,1655,0d15517e4ff3402c96a61732207af191,homeandauto,0d15517e4ff3402c96a61732207af191,male,NY,48000.0,32.0,Podcast,Desktop,49.0,1.0,True,None,2,Age: 18-34 | Income: <75k | Married: Married
902960,2155,a5fd98370b58414ba29e383d62bf1965,homeandauto,a5fd98370b58414ba29e383d62bf1965,None,NC,NaN,56.0,Podcast,Desktop,NaN,0.0,None,None,None,Unknown
887404,2155,1b7568412f794bef9a9f3e383a63aae8,homeandauto,1b7568412f794bef9a9f3e383a63aae8,None,None,NaN,NaN,Organic - Brand,Desktop,NaN,0.0,None,None,None,Unknown
18316,8007,de8ad84993334fd697c016c83bfa336e,homeandauto,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None
2146789,8007,b5eba091858e43189f570945c025c08c,homeandauto,b5eba091858e43189f570945c025c08c,female,CO,50000.0,42.0,Podcast,Desktop,21.0,0.0,False,married,6,Age: 35-54 | Income: <125k | Married: Married
459808,8745,c6e365bc8e994c1e8cd94cf9180e19ee,homeandauto,c6e365bc8e994c1e8cd94cf9180e19ee,None,None,NaN,36.0,Podcast,Desktop,NaN,0.0,None,None,None,Unknown
1444298,8745,27f5388331a595c02702fe21159af460,homeandauto,27f5388331a595c02702fe21159af460,None,CT,NaN,34.0,Organic - Brand,Desktop,NaN,0.0,None,None,None,Unknown
16898,11193,f42718d6427c4c02a523a3f8efb67e6a,homeandauto,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None
649724,11193,60b1f24b6c88efe86fbbfc9f545d8a5e,homeandauto,60b1f24b6c88efe86fbbfc9f545d8a5e,female,KY,NaN,38.0,Organic - SEO,Desktop,NaN,0.0,None,None,None,Unknown


In [42]:
## Getting our previous data
df3.shape

(13261, 9)

In [ ]:
## Merging the bigger demographic data with our smaller data

# this merge aint right, we use both application id and product type
#df_nps_map = df3.merge(df_map, how="left", on = "application_id")
#df_nps_map.shape

### Merge on application ID and product type to obtain uniqueness by application id and product type

In [ ]:
#df_nps_map.loc[df_nps_map.duplicated("application_id",keep=False)].sort_values(by="application_id").head(6)

This probably happens because the mapping table has information for all the product types a particular analytics_user_id has filled up some information for. Redo the merge on both application_id and product_type to fetch the correct analytics_user_id corresponding to each application_id. This also ensures we have the correct survey response corresponding to a particular application_id. 

In [50]:
df_nps_map2 = df3.merge(df_map, how="left", on = ["application_id","product_type"])
df_nps_map2.head()

,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim,analytics_user_id,analytics_user_id_1,gender,state_code,individual_income,age,marketing_application_source,first_touch_device,bmi,num_conditions,history_of_tobacco_use,marital_status,segment,segment_description
0,1545487.0,life,501303897,KUNeZeVDPnJBJ27M95S3oUR1JrgYPw62,2019-12-04 10:27:47,0,NaN,None,The medical exam company assured by you manage...,c91980234efd4f7f962920a3d5377fad,c91980234efd4f7f962920a3d5377fad,male,CA,150000.0,43.0,Paid Search - NB,iOS,23.0,0.0,True,single,7,Age: 35-54 | Income: >=125k | Married: Not Mar...
1,3228102.0,life,752756458,JO7J77aq77RcMybxAmLGkvBGVTDfAtzY,2020-11-12 14:23:14,0,NaN,None,You turned me down for coverage,585b27d7878640c991e736447791b943,585b27d7878640c991e736447791b943,female,TX,32280.0,71.0,Lead Gen,Desktop,24.0,1.0,False,married,11,Age: >=55 | Income: <125k | Married: All
2,1518151.0,life,445739166,lBNvqsW3t64o2lSraPhCr4YOwPVqlm5F,2019-09-05 16:18:36,0,NaN,None,"It took a very long time, more than 3 months, ...",8f5e2b05b65f4a6495e3d2dfc9d3495e,8f5e2b05b65f4a6495e3d2dfc9d3495e,female,CA,210000.0,50.0,Paid Search - NB,Desktop,30.0,3.0,True,single,7,Age: 35-54 | Income: >=125k | Married: Not Mar...
3,1516610.0,life,441389206,3IGdqgeGIZ9jXlykYKGrC9fnAnQbxkgy,2019-10-29 21:26:02,0,NaN,None,Agent was not willing to shop policy around. ...,a55aef970f774a3489f9cf07b384f86d,a55aef970f774a3489f9cf07b384f86d,male,AZ,98500.0,31.0,Partnerships,Desktop,27.0,4.0,False,married,1,Age: 18-34 | Income: >=75k | Married: Married
4,1266586.0,life,486185626,4GQA3RkvbxxxswPqAEoj8Df9SFBseCNd,2019-11-13 16:18:59,0,NaN,None,I had to reach out to Policy Genius multiple t...,f74b2400119b4c0d9894b04619b17307,f74b2400119b4c0d9894b04619b17307,male,CA,75120.0,32.0,Organic - Brand,iOS,28.0,1.0,False,married,1,Age: 18-34 | Income: >=75k | Married: Married


In [52]:
## Dimensions as required
df_nps_map2.shape

(13261, 23)

In [53]:
## looking at duplicates in application and product type
df_nps_map2.loc[df_nps_map2.duplicated(["application_id", "product_type"], keep=False)].sort_values("application_id").tail(10)


,application_id,product_type,respondent_id,survey_key,created_at,score,zendesk_ticket_id,medium,verbatim,analytics_user_id,analytics_user_id_1,gender,state_code,individual_income,age,marketing_application_source,first_touch_device,bmi,num_conditions,history_of_tobacco_use,marital_status,segment,segment_description


In [54]:
## final unique data by (application id and product type)
df_nps_map2.shape

(13261, 23)

#### OUR FINAL CLEAN DATA FOR ANALYSIS

In [55]:
Merged_final_data = df_nps_map2.copy()
Merged_final_data.shape


(13261, 23)

In [58]:
## Looking at our our final data and variables for analysis
Merged_final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13261 entries, 0 to 13260
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   application_id                13261 non-null  float64       
 1   product_type                  13261 non-null  object        
 2   respondent_id                 13261 non-null  object        
 3   survey_key                    13261 non-null  object        
 4   created_at                    13261 non-null  datetime64[ns]
 5   score                         13261 non-null  int64         
 6   zendesk_ticket_id             1086 non-null   float64       
 7   medium                        1086 non-null   object        
 8   verbatim                      13261 non-null  object        
 9   analytics_user_id             13022 non-null  object        
 10  analytics_user_id_1           13021 non-null  object        
 11  gender                      

In [67]:
Merged_final_data.to_csv('Merged_final_data.csv')

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
